In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
from settings_llama_nemo import *
from llama_nemo_rag_v2 import RAGMultiModalModel

In [6]:
import os

MODEL_NAME = f"{TEAM_ROOT_DIR}/model-weights/llama-nemoretriever-colembed-3b-v1"
SPLIT = "val"
IMAGES_DIR = os.path.abspath(os.path.expanduser(f"{TEAM_ROOT_DIR}/shared/chartqa_images/{SPLIT}"))  
INDEX_ROOT = os.path.abspath(os.path.expanduser(f"{ROOT_DIR}/.byaldi")) 
INDEX_NAME = os.path.abspath(os.path.expanduser(f"chartqa_{SPLIT}_index"))

ret = RAGMultiModalModel.from_pretrained(MODEL_NAME)
ret.index(
    input_path=IMAGES_DIR,
    index_name=INDEX_NAME,
    index_root=INDEX_ROOT,
    metadata=[{"image_name": f} for f in sorted(os.listdir(IMAGES_DIR)) if f.endswith(".png")],
    overwrite=True,
    batch_size=4,      
    max_side=512      
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/h/ws_szhang/vlm_env_s/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Extracting document embeddings...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.44s/it]


In [8]:
from pathlib import Path
from datasets import Dataset
from multi_modal_v2 import MultiModalGenerator 
from multi_modal_v2 import MultiModalRAG
import pandas as pd

ret_model_name = "llama_nemo"
gen_model_name = "qwen25_vl_7b_instruct"
generator = MultiModalGenerator(model_name=gen_model_name)
rag = MultiModalRAG(retriever=ret, generator=generator)

rag_metrics = {}

for vlm_name in ["qwen25_vl_7b_instruct", "qwen25_vl_32b_instruct", "qwen25_vl_72b_instruct_awq"]:
# for vlm_name in ["qwen25_vl_7b_instruct"]:
    refined_path = f"{TEAM_ROOT_DIR}/shared/refined_chartqa/val-100_{vlm_name}"
    if not os.path.exists(refined_path):
        continue
    refined_dset = Dataset.load_from_disk(refined_path)
    safe_dset = refined_dset.filter(lambda row: row["label"] == row["refined_label"])

    # Save the RAG Results
    save_dir = f"{TEAM_ROOT_DIR}/suzy/llama_nemo_experiment/val-100_{vlm_name}_rag_results_{ret_model_name}_{gen_model_name}"
    print(save_dir)
    
    dset, metrics = rag.run_and_evaluate_all(safe_dset, save_dir=save_dir)
    run_title = f"{len(safe_dset)} Refined QAs ({vlm_name}) - RAG ({ret_model_name}, {gen_model_name}):"
    rag_metrics[run_title] = metrics

metrics_df = pd.DataFrame(rag_metrics).transpose().multiply(100).round(2)
metrics_df

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


/projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_7b_instruct_rag_results_llama_nemo_qwen25_vl_7b_instruct
Loading RAG results from /projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_7b_instruct_rag_results_llama_nemo_qwen25_vl_7b_instruct
/projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_32b_instruct_rag_results_llama_nemo_qwen25_vl_7b_instruct


Visual RAG Inference and Evaluation:   0%|          | 0/86 [00:00<?, ?it/s]


Extracting query embeddings...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.49it/s]

Extracting query embeddings...:   0%|                                                                                                                                                      | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

Saving RAG results to /projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_32b_instruct_rag_results_llama_nemo_qwen25_vl_7b_instruct


Saving the dataset (0/1 shards):   0%|          | 0/86 [00:00<?, ? examples/s]

/projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_72b_instruct_awq_rag_results_llama_nemo_qwen25_vl_7b_instruct
Loading RAG results from /projects/multimodal_bootcamp/multimodal-td-2/suzy/llama_nemo_experiment/val-100_qwen25_vl_72b_instruct_awq_rag_results_llama_nemo_qwen25_vl_7b_instruct


,ret_precision,ret_recall,gen_correctness,gen_em,gen_precision,gen_recall
"72 Refined QAs (qwen25_vl_7b_instruct) - RAG (llama_nemo, qwen25_vl_7b_instruct):",29.63,88.89,63.89,38.89,85.42,88.89
"86 Refined QAs (qwen25_vl_32b_instruct) - RAG (llama_nemo, qwen25_vl_7b_instruct):",32.17,96.51,60.47,39.53,91.86,96.51
"95 Refined QAs (qwen25_vl_72b_instruct_awq) - RAG (llama_nemo, qwen25_vl_7b_instruct):",33.33,100.00,67.37,43.16,95.09,100.00


In [10]:
import torch
free, total = torch.cuda.mem_get_info()
print(f"free={free/1e9:.2f} GB  total={total/1e9:.2f} GB")
print(torch.cuda.memory_summary())


free=0.17 GB  total=85.10 GB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  80090 MiB |  80090 MiB |  50473 GiB |  50394 GiB |
|       from large pool |  80084 MiB |  80084 MiB |  49587 GiB |  49509 GiB |
|       from small pool |      6 MiB |     62 MiB |    885 GiB |    885 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  80090 MiB |  80090 MiB |  50473 GiB |  50394 GiB |
|       from large pool |  80084 Mi